In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab1/ai_scholar/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
import csv
import numpy as np
import nltk
import sys
from nltk.tokenize import sent_tokenize, word_tokenize
from pathlib import Path
import pickle
import re

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

Other things to try:
- dependency parsing to check questions

In [ ]:
def set_seed(seed=0):
    np.random.seed(seed)

In [ ]:
def is_good(title):
    """Check whether a title is stylish."""
    title = title.strip()
    # semicolon
    title_split = title.split(':', 1)
    flag_semicolon = False
    if len(title_split) > 1:
        # if len(word_tokenize(title_split[0])) == 1:
        title = title_split[0].strip() +' '+ title_split[1].strip()
        flag_semicolon = True
    else:
        title = title_split[0]

    # deal with specific words
    title = title.lower()
    if 'www' in title or 'journal' in title or 'volume' in title or 'conference' in title or 'copyright' in title or 'vol' in title or 'ieee' in title:
        return False

    # other punctuations
    cnt_ques, cnt_quot = 0, 0
    for idx, char in enumerate(title):
        if char == '!':
            return True
        if idx != len(title) - 1 and char == '.' and (title[idx+1] == ' ' and title[idx-1] != '.'):
            try:
                if word_tokenize(title[:idx])[-1][0].islower() and word_tokenize(title[:idx])[-2][0].islower():
                    # print('yes4')
                    return True
            except IndexError:
                pass
        if char == "'":
            flag1 = False
            flag2 = False
            if idx == 0 or (idx > 0 and title[idx-1] == ' '):
                flag1 = True
            if idx == len(title) - 1 or (idx < len(title) - 1 and title[idx+1] == ' '):
                flag2 = True
            if flag1 or flag2:
                cnt_quot += 1
        if char == '?':
            cnt_ques += 1
    if cnt_ques == 1:
        return True
    if cnt_quot == 2:
        return True
    # detect pronouns and numeric
    # https://pythonprogramming.net/part-of-speech-tagging-nltk-tutorial/
    token_list = word_tokenize(title)
    tagged_list = nltk.pos_tag(token_list)
    # print(tagged_list)
    # flag_numeric = False
    for tag_idx, tag_tuple in enumerate(tagged_list):
        if flag_semicolon and tag_idx == 0 and (tag_tuple[1] == 'TO' or tag_tuple[1][:2] == 'VB') and tag_tuple[0] != 'search':
            # print('yes1')
            return True
        elif tag_tuple[1] == 'PRP' or tag_tuple[1] == 'PRP$':
            tmp_pron = tag_tuple[0].lower()
            if tmp_pron != 's' and tmp_pron != 'it' and tmp_pron != 'its' and tmp_pron != 'i' and tmp_pron != 'their':
                # print('yes2')
                return True
        elif tag_idx == 0 and tag_tuple[1] == 'CD' and len(tag_tuple[0]) != 4:
            if tag_idx < len(tagged_list)-1 and (tag_tuple[0].isnumeric() or not tag_tuple[0][0].isnumeric()):
                tmp_str = ' '.join(token_list[tag_idx:tag_idx+4])
                if not ('=' in tmp_str or '√' in tmp_str or 'TeV' in tmp_str or 'GeV' in tmp_str or '*' in tmp_str or '/' in tmp_str or '\\' in tmp_str):
                    # print('yes3')
                    tmp_str2 = tagged_list[tag_idx+1][1][:2]
                    if tmp_str2 == 'JJ' or tmp_str2 == 'NN' or tmp_str2 == 'RB':
                        return True
                # flag_numeric = True
        elif tag_tuple[1] == ',':
            try:
                tmp_word = tagged_list[tag_idx+2][1]
                tmp_tag = tagged_list[tag_idx+1][1][:2]
                if tmp_word == 'CC' and (tmp_tag == 'NN' or tmp_tag == 'VB'):
                    return True
            except IndexError:
                pass
            try:
                tmp_word = tagged_list[tag_idx+3][1]
                tmp_tag = tagged_list[tag_idx+2][1][:2]
                if tmp_word == 'CC' and (tmp_tag == 'NN' or tmp_tag == 'VB'):
                    return True
            except IndexError:
                pass
    return False

In [ ]:
print(nltk.pos_tag(word_tokenize("Search for top squark pair production using dilepton final states in [... formula...] collision data collected at [... formula...]")))

In [ ]:
token_list = 'sgew'
tag_idx = 2
try:
  token_list[tag_idx+2]
except IndexError:
  print('yes')
print(1+2)

In [ ]:
# a quick check of the functionality
print(is_good("Search for direct production of charginos, neutralinos and sleptons in final states with two leptons and missing transverse momentum in pp collisions at = 8TeV …"))

# look at scholars that tend to write stylish titles, use 78k_scholar_new.npy

In [ ]:
scholar_all = np.load(GOOGLE_DRIVE_PATH+'gegs_scholars.npy', allow_pickle=True)

In [ ]:
path = GOOGLE_DRIVE_PATH+"author_gender.csv" #author_gender
import pandas as pd
df = pd.read_csv(path, names=('name', 'url', 'gender'))
df.insert(0, 'index', df.index)

In [ ]:
# scholar_all[0]['papers'] # a list of papers
from collections import defaultdict
sty_dict = defaultdict(float)
for scholar in scholar_all:
  for paper in scholar['papers']:
    sty_dict[scholar['url']] += int(is_good(paper[1]))
  # sty_dict[scholar['url']] /= len(scholar['papers'])

In [ ]:
import pickle
sorted_sty_dict = {k: v for k, v in sorted(sty_dict.items(), key=lambda item: item[1], reverse=True)}
with open(GOOGLE_DRIVE_PATH+'out/sorted_sty_dict1.pkl', 'wb') as file:
  pickle.dump(sorted_sty_dict, file)

In [ ]:
with open(GOOGLE_DRIVE_PATH+'out/sorted_sty_dict1.pkl', 'rb') as file:
  sorted_sty_dict = pickle.load(file)

In [ ]:
sorted_abs_sty_dict = {k: v for k, v in sorted(sty_dict.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
with open(GOOGLE_DRIVE_PATH+'out/sorted_abs_sty_dict1.pkl', 'wb') as file:
  pickle.dump(sorted_abs_sty_dict, file)

In [ ]:
with open(GOOGLE_DRIVE_PATH+'out/sorted_abs_sty_dict1.pkl', 'rb') as file:
  sorted_abs_sty_dict = pickle.load(file)

In [ ]:
scholar_all = np.load(GOOGLE_DRIVE_PATH+'general/gs_scholars_all_upd.npy', allow_pickle=True)

In [ ]:
import pandas as pd
df = pd.DataFrame.from_records(scholar_all)

In [ ]:
# tab:stattopsty
from collections import defaultdict
cnt = 0
gender_dict = defaultdict(int)
for key, value in sorted_abs_sty_dict.items():
  if cnt >= 300:
    break
  gender_dict[df.loc[df['url'] == key, 'gender'].item()] += 1
  # print(df.loc[df['url'] == key, 'gender'].item())
  if df.loc[df['url'] == key, 'gender'].item() == 'F':
    print(key)
  cnt += 1
print(gender_dict) # Priyanka Sharma

In [ ]:
tmp = 'https://scholar.google.com/citations?hl=en&user=Joq4sP8AAAAJ'

In [ ]:
sorted_abs_sty_dict[tmp]

In [ ]:
sorted_sty_dict[tmp]

In [ ]:
cnt = 0
for scholar in scholar_all:
  if scholar['url'] == tmp:
    for paper in scholar['papers']:
      if is_good(paper[1]):
        print(paper[1])
        cnt += 1
    break

In [ ]:
print(sorted_sty_dict['https://scholar.google.com/citations?hl=en&user=3FWe5OQAAAAJ'])

In [ ]:
# print statistics
cnt = 0
for key, value in sorted_abs_sty_dict.items():
  if cnt >= 10:
    break
  print(key, value)
  cnt += 1

In [ ]:
# dataframe to latex?


# use original 100k_updated_with_full_name.csv

In [ ]:
def gen_test(num_samples, seed):
    """Generate random samples."""
    set_seed(seed)
    sample_idxes = []
    csv.field_size_limit(sys.maxsize)
    with open(GOOGLE_DRIVE_PATH+'data/100k_updated_with_full_name.csv', 'r') as csvfile:
        reader_list = np.array([row['title'] for row in csv.DictReader(csvfile, delimiter=',', quotechar='"')])
        # print(reader_list)
        sample_idxes = np.random.choice(len(reader_list), size=num_samples, replace=False)
        # print(sample_idxes)
        return reader_list[sample_idxes], sample_idxes

In [ ]:
def out_test(seed):
    """Test and see random sample titles from 78k dataset."""
    reader_list, _ = gen_test(100, seed)
    # print(reader_list)
    predicts = []
    for title in reader_list:
        predicts.append(int(is_good(title)))
    with open(GOOGLE_DRIVE_PATH+f'out/new_test{seed}.csv', 'w') as outfile:
        outfile.write('title,class\n')
        outfile.write('\n'.join([f'"{reader_list[idx]}",{str(pred)}' for idx, pred in enumerate(predicts)]))
out_test(8)

In [ ]:
# cnn clickbait detector
def cnn_out_test(seed):
    reader_list, sample_idxes = gen_test(100, seed)
    with open(GOOGLE_DRIVE_PATH+f'data/cnn_clickbait_output.txt', 'r') as infile:
        total_predicts = [float(line.strip()) for line in infile.readlines()]
    predicts = []
    for idx in sample_idxes:
        if total_predicts[idx] >= 50:
            predicts.append(1)
        else:
            predicts.append(0)
    with open(GOOGLE_DRIVE_PATH+f'out/cnn_clickbait/new_test{seed}.csv', 'w') as outfile:
        outfile.write('title,class\n')
        outfile.write('\n'.join([f'"{reader_list[idx]}",{str(pred)}' for idx, pred in enumerate(predicts)]))
cnn_out_test(3)

In [ ]:
# after manual labelling, check F1 score:
def check_score(seed):
    predicts = []
    with open(GOOGLE_DRIVE_PATH+f'out/new_test{seed}.csv', 'r') as csvfile: # cnn or is_good
        reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            # print(row['title'])
            predicts.append(int(row['class']))
    if seed == 0:
        labels = [0,0,0,0,0,0,1,0,0,0,
                  0,0,0,0,0,1,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,0,
                  0,1,0,0,0,1,0,0,0,0,
                  0,0,0,1,0,0,0,0,0,0,
                  0,0,0,0,0,1,0,1,0,0,
                  0,0,1,0,0,0,0,0,0,0,
                  0,0,0,0,0,1,0,0,0,0,
                  1,0,0,0,0,0,0,0,0,0]
    elif seed == 43:
        labels = [0,0,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,0,
                  0,1,0,0,0,1,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,0,
                  0,0,1,0,0,0,0,0,0,0,
                  1,0,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,1,0,0,
                  0,0,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,0,
                  1,1,0,1,1,0,0,0,0,0]
    elif seed == 1:
        labels = [0,1,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,1,0,0,
                  0,0,0,0,1,0,0,1,1,0,
                  0,1,0,0,0,0,0,0,0,0,
                  0,1,0,0,0,0,0,0,0,1,
                  0,0,1,0,0,0,0,0,0,0,
                  0,0,0,0,1,1,0,0,0,0,
                  0,0,0,0,1,0,0,1,0,0,
                  1,0,1,0,0,0,0,0,0,0,
                  0,0,0,0,1,0,0,0,0,0]
    elif seed == 2:
        labels = [0,0,0,0,0,0,0,0,0,0,
                  0,0,1,0,0,0,0,0,1,0,
                  0,0,0,0,0,0,0,0,0,0,
                  0,0,1,0,0,0,0,0,1,0,
                  0,0,0,0,0,0,1,0,0,0,
                  0,1,0,0,0,0,1,0,1,0,
                  0,0,0,0,0,0,0,0,0,0,
                  0,0,0,1,0,0,0,0,0,1,
                  0,0,0,0,0,0,0,0,0,0,
                  0,0,1,0,0,1,0,1,0,0]
    elif seed == 3:
        labels = [0,0,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,0,
                  0,0,0,1,0,1,0,0,0,0,
                  0,0,0,1,0,0,0,0,0,0,
                  0,1,0,1,0,0,0,1,0,1,
                  0,0,1,0,0,0,0,1,0,0,
                  0,0,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,1,0,
                  0,0,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,0]
    elif seed == 4:
        labels = [0,0,1,0,0,0,0,0,0,0,
                  0,1,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,1,1,
                  0,0,0,0,0,0,1,0,0,0,
                  0,0,0,0,0,1,0,0,0,1,
                  0,0,0,0,0,0,0,1,0,1,
                  0,0,0,1,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,0]
    elif seed == 5:
        labels = [0,0,1,1,0,0,0,0,0,0,
                  1,0,0,0,0,0,0,1,0,0,
                  1,0,0,1,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,1,0,0,
                  1,0,0,0,1,1,0,0,0,0,
                  0,0,0,1,0,0,0,0,0,0,
                  1,0,0,0,0,0,0,0,0,1,
                  0,0,0,0,0,0,0,0,0,0,
                  1,0,0,1,0,0,0,0,1,0,
                  0,0,1,0,1,0,0,0,0,0,]
    elif seed == 6:
        labels = [0,0,0,0,0,0,1,1,0,0,
                  0,0,0,0,0,0,0,0,0,1,
                  0,0,0,0,0,0,1,0,0,0,
                  0,0,0,1,0,0,0,0,1,0,
                  0,0,1,0,0,0,0,0,0,0,
                  1,0,0,1,0,0,0,0,0,0,
                  0,0,0,1,0,1,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,0,
                  0,0,1,0,1,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,1,0]
    elif seed == 7:
        labels = [0,0,0,1,0,0,0,0,0,0,
                  0,0,0,1,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,1,1,0,
                  0,1,0,1,0,0,0,0,0,0,
                  1,0,0,0,0,0,0,0,0,1,
                  0,1,0,1,0,0,0,0,0,0,
                  0,0,0,0,0,1,0,0,0,1,
                  0,0,0,1,0,0,0,0,0,0,
                  0,0,0,0,1,0,0,0,0,0,
                  0,0,0,0,0,0,0,1,0,0]
    elif seed == 8:
        labels = [0,0,1,0,0,0,1,0,0,0,
                  0,0,0,0,0,1,0,0,0,0,
                  0,1,0,0,0,0,0,1,0,0,
                  0,0,0,0,0,0,0,1,0,0,
                  1,0,0,0,0,0,1,0,1,0,
                  0,0,0,1,0,1,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,0,
                  1,0,0,0,0,0,0,1,1,0,
                  0,0,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,1,0,0,0,0]
    return labels, predicts
labels, predicts = [], []
for i in (43, 0, 1, 2, 3, 4, 5, 6, 7, 8):
    labels = labels + check_score(i)[0]
    predicts = predicts + check_score(i)[1]
print('F1 Score: %.3f' % f1_score(labels, predicts))
print('Precision Score: %.3f' % precision_score(labels, predicts))
print('Recall Score: %.3f' % recall_score(labels, predicts))

the CNN model clickbait detector with bad performance

In [ ]:
def out_manual_test():
    """Test manual labeled test sets."""
    # outfile = open('data/test.csv', 'w')
    # outfile.write(',text,class,predict,question,exclamation,has_num,headline_words\n')
    # outfile.write('text,class,predict')
    cnt = 0
    labels = []
    predicts = []
    with open(GOOGLE_DRIVE_PATH+'data/test.csv', 'r') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            predict = int(is_good(row['text']))
            tmp_str = row['text']+'",'+str(row['class'])+','+str(predict)+'\n'
            # outfile.write(tmp_str)
            cnt += 1
            labels.append(int(row['class']))
            predicts.append(predict)
    # outfile.close()
    # print(predicts)
    print('F1 Score: %.3f' % f1_score(labels, predicts))
    print('Precision Score: %.3f' % precision_score(labels, predicts))
    print('Recall Score: %.3f' % recall_score(labels, predicts))
out_manual_test()

In [ ]:
def gen_final_test(infilename, outfilename):
    csv.field_size_limit(sys.maxsize)
    predicts = []
    with open(GOOGLE_DRIVE_PATH+f'data/{infilename}', 'r') as csvfile:
        for row in csv.DictReader(csvfile, delimiter=',', quotechar='"'):
            title = row['title']
            predicts.append(int(is_good(title)))
        csvfile.seek(0)
        with open(GOOGLE_DRIVE_PATH+f'out/{outfilename}', 'w') as outfile:
            writer = csv.writer(outfile)
            for idx, row in enumerate(list(csv.reader(csvfile, delimiter=',', quotechar='"'))):
                if idx == 0:
                    writer.writerow(row+['stylish_class'])
                else:
                    writer.writerow(row+[predicts[idx-1]])
gen_final_test('100k_updated_with_full_name.csv', '100k_updated_with_full_name_updated.csv')
# 100k_updated_with_full_name.csv

In [ ]:
import json
import csv
def gender_clickbaity():
    csv.field_size_limit(sys.maxsize)
    abs = []
    gender_dict = {'male':0,'female':0,'unk':0,'male_stylish':0,'female_stylish':0, 'unk_stylish':0, 'tot':0, 'tot_stylish':0}
    with open(GOOGLE_DRIVE_PATH+'out/100k_updated_with_full_name.csv', 'r') as csvfile:
        for row in csv.DictReader(csvfile, delimiter=',', quotechar='"'):
            # first author
            gender = json.loads(row['genders'].replace("'", '"'))
            clickbaity = row['stylish_class']
            if len(gender) != 0:
                gender = gender[0]
                # print(gender)
            else:
                gender_dict['unk'] += 1
                gender_dict['unk_stylish'] += int(clickbaity)
                continue
            if gender == 'M':
                gender_dict['male'] += 1
                gender_dict['male_stylish'] += int(clickbaity)
                gender_dict['tot_stylish'] += int(clickbaity)
            elif gender == 'F':
                gender_dict['female'] += 1
                gender_dict['female_stylish'] += int(clickbaity)
                gender_dict['tot_stylish'] += int(clickbaity)
            elif gender == '-':
                gender_dict['unk'] += 1
                gender_dict['unk_stylish'] += int(clickbaity)
                gender_dict['tot_stylish'] += int(clickbaity)
                if int(clickbaity) == 1:
                    abs.append(row['']+'|"'+row['abstract']+'"')
            else:
                raise ValueError(gender)
            gender_dict['tot'] += 1
        return gender_dict, abs
gender_dict, abs = gender_clickbaity()
gender_dict

In [ ]:
import json
import csv
def gender_clickbaity():
    csv.field_size_limit(sys.maxsize)
    abs = []
    gender_dict = {'male':0,'female':0,'unk':0,'male_stylish':0,'female_stylish':0, 'unk_stylish':0, 'tot':0, 'tot_stylish':0}
    with open(GOOGLE_DRIVE_PATH+'out/100k_updated_with_full_name.csv', 'r') as csvfile:
        for row in csv.DictReader(csvfile, delimiter=',', quotechar='"'):
            # first author
            gender = json.loads(row['genders'].replace("'", '"'))
            clickbaity = row['stylish_class']
            if len(gender) != 0:
                gender = gender[0]
                # print(gender)
            else:
                gender_dict['unk'] += 1
                gender_dict['unk_stylish'] += int(clickbaity)
                continue
            if gender == 'M':
                gender_dict['male'] += 1
                gender_dict['male_stylish'] += int(clickbaity)
                gender_dict['tot_stylish'] += int(clickbaity)
            elif gender == 'F':
                gender_dict['female'] += 1
                gender_dict['female_stylish'] += int(clickbaity)
                gender_dict['tot_stylish'] += int(clickbaity)
            elif gender == '-':
                gender_dict['unk'] += 1
                gender_dict['unk_stylish'] += int(clickbaity)
                gender_dict['tot_stylish'] += int(clickbaity)
                if int(clickbaity) == 1:
                    abs.append(row['']+'|"'+row['abstract']+'"')
            else:
                raise ValueError(gender)
            gender_dict['tot'] += 1
        return gender_dict, abs
gender_dict, abs = gender_clickbaity()
gender_dict

Write the abstracts of paper by authors with unclassified genders to `out/unclassified_abstract.csv`

In [ ]:
with open(GOOGLE_DRIVE_PATH+'out/unclassified_abstract.csv', 'w') as csvfile:
        csvfile.write('\n'.join(abs))

In [ ]:
print('m_avg: {}'.format(gender_dict['male_stylish']/gender_dict['male']))
print('f_avg: {}'.format(gender_dict['female_stylish']/gender_dict['female']))
print('avg: {}'.format(gender_dict['tot_stylish']/gender_dict['tot']))

Detect the language of the unclassified abstracts

In [ ]:
!pip install spacy_langdetect

In [ ]:
# spacy detect language of longer sentences
from spacy_langdetect import LanguageDetector
import spacy
nlp = spacy.load('en')
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

import pandas as pd
from collections import defaultdict
lang = defaultdict(int)
with open(GOOGLE_DRIVE_PATH+f'out/unclassified_abstract.csv', 'r') as csvfile:
  for row in csv.reader(csvfile, delimiter='|', quotechar='"'):
    try:
      lang[nlp(row[1])._.language['language']] += 1
    except IndexError as e:
      print(e)
      continue

In [ ]:
lang

# simply for test

In [ ]:
import pickle
csv.field_size_limit(sys.maxsize)
with open(GOOGLE_DRIVE_PATH+f'out/100k_updated_with_full_name.csv', 'r') as csvfile:
    cnt = 0
    for row in csv.DictReader(csvfile, delimiter=',', quotechar='"'):
      if cnt < 10:
        print(row['abstract'])
      else:
        break
      cnt += 1